In [1]:
import nltk
import pandas as pd
import json

In [3]:
data = []

In [4]:
with open('113.txt') as json_data:
    d = json.load(json_data)
    for i in d:
        data.append(i)
    json_data.close()

In [5]:
len(data)

16586

In [6]:
df = pd.DataFrame(data)

In [22]:
df.shape

(16586, 6)

In [24]:
df=df.drop_duplicates('speech')

In [25]:
df.shape

(11565, 6)

In [7]:
df

,date,session,speaker,speech,title,url
0,2013-01-03,113,VICE,PRESIDENT. The Chaplain will lead the Senate i...,,http://www.congress.gov/congressional-record/2...
1,2013-01-03,113,VICE,"PRESIDENT led the pledge of allegiance, as fol...",PLEDGE OF ALLEGIANCE,http://www.congress.gov/congressional-record/2...
2,2013-01-03,113,VICE,PRESIDENT. The majority leader is recognized.,RECOGNITION OF THE MAJORITY LEADER,http://www.congress.gov/congressional-record/2...
3,2013-01-03,113,REID,"Mr. President, I have a number of housekeeping...",ORDER OF BUSINESS,http://www.congress.gov/congressional-record/2...
4,2013-01-03,113,VICE,PRESIDENT. The clerk will report the resolutio...,INFORMING THE PRESIDENT OF THE UNITED STATES T...,http://www.congress.gov/congressional-record/2...
5,2013-01-03,113,McCONNELL,"Mr. President, I move to lay that motion on th...",INFORMING THE PRESIDENT OF THE UNITED STATES T...,http://www.congress.gov/congressional-record/2...
6,2013-01-03,113,PRESIDENT,"pro tempore. Pursuant to S. Res. 1, the Chair ...",INFORMING THE PRESIDENT OF THE UNITED STATES T...,http://www.congress.gov/congressional-record/2...
7,2013-01-03,113,REID,"Mr. President, I move to reconsider the vote b...",INFORMING THE PRESIDENT OF THE UNITED STATES T...,http://www.congress.gov/congressional-record/2...
8,2013-01-03,113,McCONNELL,I move to lay that motion on the table. The m...,INFORMING THE HOUSE OF REPRESENTATIVES THAT A ...,http://www.congress.gov/congressional-record/2...
9,2013-01-03,113,PRESIDENT,"pro tempore. Without objection, the resolution...",INFORMING THE HOUSE OF REPRESENTATIVES THAT A ...,http://www.congress.gov/congressional-record/2...


In [46]:
unique_titles= df.title.unique()

In [47]:
len(unique_titles)

3539

http://brandonrose.org/clustering

In [26]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [27]:
titles = df.title
speeches = df.speech
speakers = df.speaker

In [55]:
stopwords = nltk.corpus.stopwords.words('english')

In [57]:
morewords = ['president', 'senate', 'mr', 'mrs', 'miss', 'congress']
stopwords = stopwords+morewords

In [29]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [30]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [31]:

#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in speeches:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [32]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 1460159 items in vocab_frame


In [78]:
print(vocab_frame.head())

              words
presid    president
the             the
chaplain   chaplain
will           will
lead           lead


In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(speeches) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 54.6 s, sys: 694 ms, total: 55.3 s
Wall time: 56.9 s
(11565, 13)


In [70]:
terms = tfidf_vectorizer.get_feature_names()

In [71]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [72]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 3.82 s, sys: 37.7 ms, total: 3.86 s
Wall time: 3.98 s


In [73]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [74]:
len(clusters)

11565

In [75]:
df['clusters']=clusters

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [76]:
df['clusters'].value_counts()

1    3854
0    2644
3    1950
4    1560
2    1557
Name: clusters, dtype: int64

In [77]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
#     print("Cluster %d titles:" % i, end='')
#     for title in df.ix[i]['title'].values.tolist():
#         print(' %s,' % title, end='')
#     print() #add whitespace
#     print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: b'amendments', b'bills', b's.', b'committee', b'ordered', b'mr.',

Cluster 1 words: b'senate', b'president', b'ask', b'mr.', b'mr.', b'committee',

Cluster 2 words: b'officer', b'ordered', b'senate', b"'s", b's.', b'states',

Cluster 3 words: b"'s", b'states', b'president', b'mr.', b'mr.', b'senate',

Cluster 4 words: b'mr.', b's.', b'committee', b'bills', b'states', b'amendments',





In [63]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(random_state=111)


In [64]:
dbscan

DBSCAN(algorithm='auto', eps=0.5, leaf_size=30, metric='euclidean',
    min_samples=5, p=None, random_state=111)

In [ ]:
dbscan.fit()

In [65]:
byspeaker=df.groupby('speaker')

In [66]:
byspeaker